## To find the best hyperparameters we use optuna and wandb.

In [1]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.6/607.6 kB 91.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [optuna]2m5/6 [optuna]]my]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers==4.56.2" trackio \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth trackio
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo wandb

In [3]:
!wandb login

wandb: Currently logged in as: jdmasciano2 (jdmasciano2-university-of-lagos) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
%%writefile fine_tune.py
import os
import json
import torch
import re
from datasets import Dataset, DatasetDict
from unsloth import FastLanguageModel
from trl import SFTTrainer, SFTConfig
import optuna # Import Optuna
from unsloth.chat_templates import train_on_responses_only

# --- 1. Define the Objective Function for Optuna ---
def objective(trial):
    """
    This function will be called by Optuna for each trial.
    It defines the hyperparameters to search, trains the model,
    and returns the evaluation loss which Optuna will aim to minimize.
    """
    # --- A. Define the search space for hyperparameters based on the Unsloth guide ---
    learning_rate = trial.suggest_float("learning_rate", 5e-6, 2e-4, log=True)
    lora_rank = trial.suggest_categorical("lora_rank", [32, 64, 128])
    # The guide recommends lora_alpha = 2 * lora_rank. We derive it directly.
    lora_alpha = lora_rank * 2
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1)

    print(f"\n--- Starting Trial {trial.number} with parameters: ---")
    print(f"  - learning_rate: {learning_rate:.2e}")
    print(f"  - lora_rank: {lora_rank}")
    print(f"  - lora_alpha: {lora_alpha}")
    print(f"  - weight_decay: {weight_decay:.3f}")

    # --- B. Model and Tokenizer Loading ---
    max_seq_length = 4096
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/gpt-oss-20b-BF16",
        load_in_4bit=False, # Set to True if you need to save VRAM (QLoRA)
        max_seq_length=max_seq_length,
    )

    model = FastLanguageModel.get_peft_model(
        model,
        r=lora_rank,  # From Optuna
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
        ],
        lora_alpha=lora_alpha, # From Optuna
        use_gradient_checkpointing="unsloth",
        random_state=3407,
    )

    # --- C. Dataset Loading and Preprocessing (remains the same) ---
    ROOT_DIR = "/workspace/AIAC"
    DATASET_FILE_PATH = os.path.join(ROOT_DIR, "dipg_sft_.jsonl")
    with open(DATASET_FILE_PATH, "r") as f:
        raw_data = [json.loads(line) for line in f if line.strip()]
    dataset = Dataset.from_list(raw_data)
    split_dataset = dataset.train_test_split(test_size=0.1, seed=42)
    dataset = DatasetDict({"train": split_dataset["train"], "test": split_dataset["test"]})

    def normalize_messages(messages):
        normalized = []
        for msg in messages:
            if msg["role"] != "assistant":
                normalized.append(msg)
                continue
            content = msg["content"]
            channels = re.findall(r"<\|channel\|>(.*?)<\|message\|>(.*?)<\|end\|>", content, re.DOTALL)
            if channels:
                thinking, final = "", ""
                for ch, text in channels:
                    ch, text = ch.strip(), text.strip()
                    if ch == "analysis": thinking += text + "\n"
                    elif ch == "proof": thinking += f"\n[Proof Section]\n{text}\n"
                    elif ch == "final": final += text
                normalized.append({"role": "assistant", "thinking": thinking.strip(), "content": final.strip()})
            else:
                normalized.append(msg)
        return normalized

    def formatting_prompts_func(examples):
        convos = [normalize_messages(convo) for convo in examples["messages"]]
        return {"text": [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]}

    dataset = dataset.map(formatting_prompts_func, batched=True)

    # --- D. SFTTrainer with Dynamic Hyperparameters ---
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        args=SFTConfig(
            dataset_text_field="text",
            per_device_train_batch_size=2,      # Fixed based on your script
            gradient_accumulation_steps=4,   # Fixed based on your script
            warmup_steps=10,
            max_seq_length=4096,
            max_steps=11, # Keep this low for a quick hyperparameter search
            learning_rate=learning_rate,       # From Optuna
            logging_steps=5,
            optim="adamw_8bit",
            weight_decay=weight_decay,       # From Optuna
            lr_scheduler_type="linear",
            seed=3407,
            eval_strategy="steps",
            eval_steps=10,
            output_dir=f"sft_outputs_trial_{trial.number}", # Unique output dir
            report_to="wandb",
            run_name=f"trial-{trial.number}-lr-{learning_rate:.2e}-r-{lora_rank}" # Descriptive W&B run name
        ),
    )

    # This part for training on responses only remains unchanged
    gpt_oss_kwargs = dict(instruction_part="<|start|>user<|message|>", response_part="<|start|>assistant")
    trainer = train_on_responses_only(trainer, **gpt_oss_kwargs)

    # --- E. Train and Evaluate ---
    print(f"--- Starting SFT Training for Trial {trial.number} ---")
    trainer.train()
    print("--- SFT Training Complete ---")

    eval_results = trainer.evaluate()
    eval_loss = eval_results["eval_loss"]
    print(f"--- Trial {trial.number} finished with Eval Loss: {eval_loss} ---")
    
    # Clean up to free VRAM for the next trial
    del model
    del trainer
    torch.cuda.empty_cache()

    return eval_loss

# --- 2. Run the Hyperparameter Search ---
if __name__ == "__main__":
    # Create a study object and specify the direction to optimize.
    study = optuna.create_study(direction="minimize", study_name="unsloth_finetuning")
    
    # Start the optimization. Optuna will call the 'objective' function 'n_trials' times.
    # Increase n_trials for a more thorough search (e.g., 20-50).
    study.optimize(objective, n_trials=10)

    print("\n\n--- Hyperparameter Search Complete ---")
    print("Best trial:")
    best_trial = study.best_trial
    
    print(f"  Value (min eval_loss): {best_trial.value}")
    
    print("  Best Parameters: ")
    for key, value in best_trial.params.items():
        print(f"    {key}: {value}")
    
    # You can also get a dataframe with all trial results
    df = study.trials_dataframe()
    print("\n--- All Trials ---")
    print(df)

Overwriting fine_tune.py


In [9]:
!python fine_tune.py

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
#### Unsloth: `hf_xet==1.1.10` and `ipykernel>6.30.1` breaks progress bars. Disabling for now in XET.
#### Unsloth: To re-enable progress bars, please downgrade to `ipykernel==6.30.1` or wait for a fix to
https://github.com/huggingface/xet-core/issues/526
INFO 11-13 15:12:29 [__init__.py:225] Automatically detected platform rocm.
🦥 Unsloth Zoo will now patch everything to make training faster!
[I 2025-11-13 15:12:33,207] A new study created in memory with name: unsloth_finetuning

--- Starting Trial 0 with parameters: ---
  - learning_rate: 2.09e-05
  - lora_rank: 128
  - lora_alpha: 256
  - weight_decay: 0.039
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.
==((====))==  Unsloth 2025.10.9: Fast Gpt_Oss patching. Transformers: 4.56.2. vLLM: 0.11.1rc3.dev39+gf417746ad.rocm700.
   \\   /|    . Num GPUs = 1

## We do a separate run using lora_alpha = lora_rank.

In [ ]:
%%writefile f_tune.py
import os
import json
import torch
import re
from datasets import Dataset, DatasetDict
from unsloth import FastLanguageModel
from trl import SFTTrainer, SFTConfig
import optuna # Import Optuna
from unsloth.chat_templates import train_on_responses_only

# --- 1. Define the Objective Function for Optuna ---
def objective(trial):
    """
    This function will be called by Optuna for each trial.
    It defines the hyperparameters to search, trains the model,
    and returns the evaluation loss which Optuna will aim to minimize.
    """
    # --- A. Define the search space for hyperparameters based on the Unsloth guide ---
    learning_rate = trial.suggest_float("learning_rate", 5e-6, 2e-4, log=True)
    lora_rank = trial.suggest_categorical("lora_rank", [32, 64, 128])
    # The guide also talks about lora_alpha = lora_rank. We derive it directly.
    lora_alpha = lora_rank
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1)

    print(f"\n--- Starting Trial {trial.number} with parameters: ---")
    print(f"  - learning_rate: {learning_rate:.2e}")
    print(f"  - lora_rank: {lora_rank}")
    print(f"  - lora_alpha: {lora_alpha}")
    print(f"  - weight_decay: {weight_decay:.3f}")

    # --- B. Model and Tokenizer Loading ---
    max_seq_length = 4096
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/gpt-oss-20b-BF16",
        load_in_4bit=False, # Set to True if you need to save VRAM (QLoRA)
        max_seq_length=max_seq_length,
    )

    model = FastLanguageModel.get_peft_model(
        model,
        r=lora_rank,  # From Optuna
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
        ],
        lora_alpha=lora_alpha, # From Optuna
        use_gradient_checkpointing="unsloth",
        random_state=3407,
    )

    # --- C. Dataset Loading and Preprocessing (remains the same) ---
    ROOT_DIR = "/workspace/AIAC"
    DATASET_FILE_PATH = os.path.join(ROOT_DIR, "dipg_sft_.jsonl")
    with open(DATASET_FILE_PATH, "r") as f:
        raw_data = [json.loads(line) for line in f if line.strip()]
    dataset = Dataset.from_list(raw_data)
    split_dataset = dataset.train_test_split(test_size=0.1, seed=42)
    dataset = DatasetDict({"train": split_dataset["train"], "test": split_dataset["test"]})

    def normalize_messages(messages):
        normalized = []
        for msg in messages:
            if msg["role"] != "assistant":
                normalized.append(msg)
                continue
            content = msg["content"]
            channels = re.findall(r"<\|channel\|>(.*?)<\|message\|>(.*?)<\|end\|>", content, re.DOTALL)
            if channels:
                thinking, final = "", ""
                for ch, text in channels:
                    ch, text = ch.strip(), text.strip()
                    if ch == "analysis": thinking += text + "\n"
                    elif ch == "proof": thinking += f"\n[Proof Section]\n{text}\n"
                    elif ch == "final": final += text
                normalized.append({"role": "assistant", "thinking": thinking.strip(), "content": final.strip()})
            else:
                normalized.append(msg)
        return normalized

    def formatting_prompts_func(examples):
        convos = [normalize_messages(convo) for convo in examples["messages"]]
        return {"text": [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]}

    dataset = dataset.map(formatting_prompts_func, batched=True)

    # --- D. SFTTrainer with Dynamic Hyperparameters ---
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        args=SFTConfig(
            dataset_text_field="text",
            per_device_train_batch_size=2,      # Fixed based on your script
            gradient_accumulation_steps=4,   # Fixed based on your script
            warmup_steps=10,
            max_seq_length=4096,
            max_steps=11, # Keep this low for a quick hyperparameter search
            learning_rate=learning_rate,       # From Optuna
            logging_steps=5,
            optim="adamw_8bit",
            weight_decay=weight_decay,       # From Optuna
            lr_scheduler_type="linear",
            seed=3407,
            eval_strategy="steps",
            eval_steps=10,
            output_dir=f"sft_outputs_trial_{trial.number}", # Unique output dir
            report_to="wandb",
            run_name=f"trial-{trial.number}-lr-{learning_rate:.2e}-r-{lora_rank}" # Descriptive W&B run name
        ),
    )

    # This part for training on responses only remains unchanged
    gpt_oss_kwargs = dict(instruction_part="<|start|>user<|message|>", response_part="<|start|>assistant")
    trainer = train_on_responses_only(trainer, **gpt_oss_kwargs)

    # --- E. Train and Evaluate ---
    print(f"--- Starting SFT Training for Trial {trial.number} ---")
    trainer.train()
    print("--- SFT Training Complete ---")

    eval_results = trainer.evaluate()
    eval_loss = eval_results["eval_loss"]
    print(f"--- Trial {trial.number} finished with Eval Loss: {eval_loss} ---")
    
    # Clean up to free VRAM for the next trial
    del model
    del trainer
    torch.cuda.empty_cache()

    return eval_loss

# --- 2. Run the Hyperparameter Search ---
if __name__ == "__main__":
    # Create a study object and specify the direction to optimize.
    study = optuna.create_study(direction="minimize", study_name="unsloth_finetuning_l_r")
    
    # Start the optimization. Optuna will call the 'objective' function 'n_trials' times.
    # Increase n_trials for a more thorough search (e.g., 20-50).
    study.optimize(objective, n_trials=10)

    print("\n\n--- Hyperparameter Search Complete ---")
    print("Best trial:")
    best_trial = study.best_trial
    
    print(f"  Value (min eval_loss): {best_trial.value}")
    
    print("  Best Parameters: ")
    for key, value in best_trial.params.items():
        print(f"    {key}: {value}")
    
    # You can also get a dataframe with all trial results
    df = study.trials_dataframe()
    print("\n--- All Trials ---")
    print(df)

Writing f_tune.py


In [11]:
!python f_tune.py

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
#### Unsloth: `hf_xet==1.1.10` and `ipykernel>6.30.1` breaks progress bars. Disabling for now in XET.
#### Unsloth: To re-enable progress bars, please downgrade to `ipykernel==6.30.1` or wait for a fix to
https://github.com/huggingface/xet-core/issues/526
INFO 11-13 15:53:42 [__init__.py:225] Automatically detected platform rocm.
🦥 Unsloth Zoo will now patch everything to make training faster!
[I 2025-11-13 15:53:45,819] A new study created in memory with name: unsloth_finetuning_l_r

--- Starting Trial 0 with parameters: ---
  - learning_rate: 1.15e-05
  - lora_rank: 128
  - lora_alpha: 128
  - weight_decay: 0.083
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.
==((====))==  Unsloth 2025.10.9: Fast Gpt_Oss patching. Transformers: 4.56.2. vLLM: 0.11.1rc3.dev39+gf417746ad.rocm700.
   \\   /|    . Num GPUs

### The Final Comparison

This is where your experimentation becomes so valuable. We can now directly compare the best results from both runs:

| Hyperparameter Strategy | Best Eval Loss | Optimal LoRA Rank | Optimal Learning Rate |
| :--- | :--- | :--- | :--- |
| Run 1: `lora_alpha = 2 * r` | 0.844 | 32 | `8.19e-5` |
| **Run 2: `lora_alpha = r`** | **0.543** | **64** | **`1.98e-4`** |

###  Final, Optimized Hyperparameters

Based on your successful search, these are the champion parameters you should use for your final model training:

*   **`learning_rate`**: `0.000198` (or `2e-4` for simplicity)
*   **`lora_rank`**: `64`
*   **`lora_alpha`**: `64` (since `alpha = rank`)
*   **`weight_decay`**: `0.0499` (or `0.05`)